## Model Development

In [1]:
import pandas as pd
import skfuzzy as fuzz
import numpy as np
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt
from ipynb.fs.full.weather_data_preprocess import data, train_data, test_data

In [2]:
class Fuzzify(object):
    def __init__(self, data_series, Min, Max, Mid):
        """Create a triangular membership function with a data series and its max, min and median"""
        
        self.data = data_series
        self.low = Min
        self.high = Max
        self.mid = Mid
        self.universe = np.arange(np.floor(self.low), np.ceil(self.high)+0.2, 0.1)
        self.trimf_low  = fuzz.trimf(self.universe, [self.low, self.low, self.mid])
        self.trimf_mid = fuzz.trimf(self.universe, [self.low, self.mid, self.high+0.1])
        self.trimf_hi  =  fuzz.trimf(self.universe, [self.mid, self.high+0.1, self.high+0.1])
        
    def get_universe(self):
        return self.universe
        
    def get_membership(self):
        """Assign fuzzy membership to each observation in the data series"""
        
        new_df = pd.DataFrame(self.data)
        new_df['low'] = fuzz.interp_membership(self.universe, self.trimf_low, self.data)
        new_df['mid'] = fuzz.interp_membership(self.universe, self.trimf_mid, self.data)
        new_df['high'] = fuzz.interp_membership(self.universe, self.trimf_hi, self.data)
        new_df['membership'] = new_df.loc[:, ['low', 'mid', 'high']].idxmax(axis = 1)
        new_df['degree'] = new_df.loc[:, ['low', 'mid', 'high']].max(axis = 1)
        return new_df

In [8]:
#Fuzzifying the relative humidity data

humidity = train_data.iloc[:,0]

humidity_min = humidity.min()
humidity_max =  humidity.max()                   
humidity_mid =  np.median(np.arange(humidity_min, humidity_max, 0.1))            

humidity_object = Fuzzify(humidity, humidity_min, humidity_max, humidity_mid)
fuzzified_humidity = humidity_object.get_membership()
fuzzified_humidity.head(5)

,humidity,low,mid,high,membership,degree
6101,74.0,0.245283,0.754717,0.000000,mid,0.754717
9966,76.0,0.000000,0.993865,0.006135,mid,0.993865
14034,79.0,0.000000,0.625767,0.374233,mid,0.625767
21924,77.0,0.000000,0.871166,0.128834,mid,0.871166
14190,76.0,0.000000,0.993865,0.006135,mid,0.993865


In [7]:
#Fuzzifying the temperature data

temperature = train_data.iloc[:,1]
temperature_min =  temperature.min()
temperature_max =  temperature.max()                    
temperature_mid =  np.median(np.arange(temperature_min, temperature_max, 0.1))               

temperature_object = Fuzzify(temperature, temperature_min, temperature_max, temperature_mid)
fuzzified_temperature = temperature_object.get_membership()
fuzzified_temperature.head(5)

,temperature,low,mid,high,membership,degree
6101,25.0,0.000000,0.666667,0.333333,mid,0.666667
9966,25.0,0.000000,0.666667,0.333333,mid,0.666667
14034,25.0,0.000000,0.666667,0.333333,mid,0.666667
21924,24.0,0.310345,0.689655,0.000000,mid,0.689655
14190,25.0,0.000000,0.666667,0.333333,mid,0.666667


In [9]:
#Fuzzifing the heat index data

heat = train_data.iloc[:,2]
heat_min =  heat.min()
heat_max =  heat.max()                    
heat_mid =  np.median(np.arange(heat_min, heat_max, 0.1))              

heat_object = Fuzzify(heat, heat_min, heat_max, heat_mid)
fuzzified_heat = heat_object.get_membership()

fuzzified_heat.head(5)

,heat,low,mid,high,membership,degree
6101,25.5,0.000000e+00,0.222222,0.777778,high,0.777778
9966,25.5,0.000000e+00,0.222222,0.777778,high,0.777778
14034,25.6,0.000000e+00,0.148148,0.851852,high,0.851852
21924,24.5,9.270362e-15,0.962963,0.037037,mid,0.962963
14190,25.5,0.000000e+00,0.222222,0.777778,high,0.777778


In [11]:
def get_rule(train_data, *arg):
    """ return the final fuzzy rule given any number of input data columns"""
    
    rule_df = train_data.copy()
    rule_df['degree'] = np.ones(train_data.shape[0])
    for col in rule_df.columns[:-1]:
        idx = train_data.columns.get_loc(col)
        rule_df[col] = arg[idx]['membership']
        rule_df['degree'] *=  arg[idx]['degree']
    final_rule = rule_df.groupby(list(rule_df.columns[:-2])).max()
    final_rule = final_rule.reset_index()
    return final_rule

get_rule(train_data, fuzzified_humidity, fuzzified_temperature, fuzzified_heat)

,humidity,temperature,heat,degree
0,high,low,low,0.815951
1,high,mid,mid,0.605505
2,low,mid,mid,0.631005
3,mid,high,high,0.503286
4,mid,low,low,0.993865
5,mid,mid,mid,0.655623
